In [1]:
# 제목 전부 (썽공)
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup
import time

all_title = []
previous_titles = None
date_range = pd.date_range(start="2023-08-31", end="2024-08-31").strftime('%Y%m%d')

for date in date_range:
    page = 1
    while True:
        res = rq.get(f'https://news.daum.net/breakingnews/society/education?page={page}&regDate={date}', headers={
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'
        })
        soup = BeautifulSoup(res.text, 'html.parser')
        titles = soup.select('#mArticle > div.box_etc > ul > li:nth-child(-n+15) > div > strong > a')
        next_button = soup.select_one('#mArticle > div.box_etc > div > span > a.btn_page.btn_next > span')
        current_titles = [title.text for title in titles]

        if not current_titles and not next_button:
            break

        if previous_titles == current_titles:
            break

        all_title.extend(current_titles)
        previous_titles = current_titles

        page += 1
        time.sleep(1)

Processing_2:  37%|███▋      | 130592/351323 [8:26:37<6:29:50,  9.44it/s]     

In [ ]:
# 언론사 및 링크 (성공)
import requests as rq
from bs4 import BeautifulSoup
import time

all_Name = []
all_links = []
previous_titles = None
date_range = pd.date_range(start="2023-08-31", end="2024-08-31").strftime('%Y%m%d')

for date in date_range:
    page = 1
    while True:
        res = rq.get(f'https://news.daum.net/breakingnews/society/affair?page={page}&regDate={date}', headers={
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'
        })
        soup = BeautifulSoup(res.text, 'html.parser')
        titles = soup.select('#mArticle > div.box_etc > ul > li:nth-child(-n+15) > div > strong > a')
        Names = soup.select('span.info_news')
        next_button = soup.select_one('#mArticle > div.box_etc > div > span > a.btn_page.btn_next > span')
        current_titles = [title.text for title in titles]
        links = soup.select('#mArticle .box_etc strong > a')

        if not current_titles and not next_button:
            break

        if previous_titles == current_titles:
            break

        for j in range(len(Names)):
            all_Name.append(Names[j].contents[0])

        for z in range(len(links)):
            all_links.append(links[z]['href'])

        previous_titles = current_titles
        page += 1
        time.sleep(1)

In [ ]:
# 기자, 날짜, 내용 (성공)
import requests as rq
from bs4 import BeautifulSoup
import concurrent.futures
from tqdm import tqdm
import re

all_Reporter = []
all_Date = []
all_content = []

def fetch_article_data(link):
    try:
        res_2 = rq.get(link, headers={
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'
        })
        soup_2 = BeautifulSoup(res_2.text, 'html.parser')
        Reporter_2 = soup_2.select('div.info_view > .txt_info')
        Dates_2 = soup_2.select('div.info_view > span > .num_date')
        contents_2 = soup_2.select('#mArticle > div.news_view.fs_type1')
        Reporter_text = Reporter_2[0].text
        Dates_text = Dates_2[0].text
        contents_3 = re.sub(r'\s+', ' ', contents_2[0].text.replace("\n", "").strip())
        return (Reporter_text, Dates_text, contents_3)
    
    except:
        pass

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    results = list(tqdm(executor.map(fetch_article_data, all_links), total=len(all_links)))

for result in results:
    all_Reporter.append(result[0])
    all_Date.append(result[1])
    all_content.append(result[2])

In [ ]:
df = pd.DataFrame({
    'pub_name' : all_Name,
    'title' : all_title,
    'author' : all_Reporter,
    'publication_date' : all_Date,
    'content' : all_content,
    'article_url': all_links
})

In [ ]:
# 스티커 드디어 완성 (성공)

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import re

all_Sticker = []

def process_link(link):
    driver = webdriver.Chrome()
    driver.get(link)
    try:
        sticker_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="alex_action_emotion"]'))
        )
        recommend_text = re.sub(r"(\d)([가-힣])", r"\1 \2", sticker_element.text.replace("\n", ""))
        return recommend_text
    
    except:
        pass

    finally:
        driver.quit()

with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(process_link, link) for link in all_links]
    
    for future in tqdm(futures, desc="Processing"):
        try:
            result = future.result()
            if result:
                all_Sticker.append(result)
        except:
            pass

In [8]:
#%pip install requests
#%pip install beautifulsoup4
#%pip install tqdm
#%pip install pandas

  Using cached pandas-2.0.3-cp38-cp38-win_amd64.whl.metadata (18 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl.metadata (5.6 kB)
Using cached pandas-2.0.3-cp38-cp38-win_amd64.whl (10.8 MB)
Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl (14.9 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Note: you may need to restart the kernel to use updated packages.
